# **Deforum Colab Notebook (v0.9)**

## Setup

In [ ]:
#@markdown **Environment Setup**
import os
import subprocess
import sys
import time

def setup_environment():
    # Check if in google colab
    try:
        ipy = get_ipython()
        if 'google.colab' not in str(ipy):
            print('Not running on Google Colab. Environment setup aborted.\n')
            print('```')
            print('Environment: Non Google Colab')
            print('Error: This notebook is not running in Google Colab.')
            print('```')
            print("Please ensure this notebook runs in Google Colab.")
            print("If this problem persists, please open an issue at: https://github.com/deforum-studio/deforum/issues")
            return
    except Exception:
        print("Error! Not running in an IPython environment. Environment setup aborted.\n")
        print('```')
        print('Environment: Non IPython')
        print('Error: This notebook is not running in an IPython environment.')
        print('```')
        print("If this problem persists, please open an issue at: https://github.com/deforum-studio/deforum/issues")
        return

    # Start the timer
    start_time = time.time()

    # Specify requirements in a list for easy editing
    requirements = ['einops', 'timm', 'basicsr', 'accelerate', 'transformers', 'torchsde']

    try:
        # Clone the deforum repo
        process = subprocess.Popen(['git', 'clone', 'https://github.com/deforum-studio/deforum.git'])
        process.wait()

        # Install required packages in one go
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', *requirements])

        # Append the path
        sys.path.append('./deforum/src/')
    except Exception as e:
        print("An error occurred during the setup.\n")
        print('```')
        print('Error:', str(e))
        print('```')
        print("Please copy the above text and open an issue at: https://github.com/deforum-studio/deforum/issues")
        return

    # Report the total time
    total_time = time.time() - start_time
    print('Environment setup completed successfully. Total time: {:.2f} seconds'.format(total_time))

setup_environment()

# import deforum
try:
    from deforum import DeforumAnimationPipeline
except Exception as e:
    print("An error occurred during the import.")
    print('```')
    print('Error:', str(e))
    print('```')
    print("Please copy the above text and open an issue at: https://github.com/deforum-studio/deforum/issues")

In [ ]:
#@markdown **Path Setup**
def path_setup():
    # Start the timer
    start_time = time.time()
    try:
        models_path = "models" #@param {type:"string"}
        output_path = "output" #@param {type:"string"}
        mount_google_drive = True #@param {type:"boolean"}
        force_remount = False
        models_path_gdrive = "/content/drive/MyDrive/AI/models" #@param {type:"string"}
        output_path_gdrive = "/content/drive/MyDrive/AI/output" #@param {type:"string"}

        print("Setting up paths...")

        try:
            ipy = get_ipython()

            if 'google.colab' in str(ipy):
                print("Executing in Google Colab...")

                if mount_google_drive:
                    print("Mounting Google Drive...")

                    from google.colab import drive # type: ignore
                    try:
                        drive_path = "/content/drive"
                        drive.mount(drive_path,force_remount=force_remount)
                        models_path = models_path_gdrive
                        output_path = output_path_gdrive

                        print("Mounted Google Drive successfully.")
                    except:
                        print("Error mounting drive or with drive path variables.")
                        print("Reverting to default path variables.")

        except:
            print("Executing in a local environment...")
            print("Using default path variables.")

    except Exception as e:
        print(f"An unexpected error occurred during the path setup: {str(e)}")
        print("If you encounter any issues, please copy this console output and open an issue at: https://github.com/deforum-studio/deforum/issues")

    finally:
        print(f"Models will be saved in this path: {models_path}")
        print(f"Output will be saved in this path: {output_path}")

    # Report the total time
    total_time = time.time() - start_time
    print('Path setup completed successfully. Total time: {:.2f} seconds'.format(total_time))
    return models_path, output_path

models_path, output_path = path_setup()

In [ ]:
import os, re
from urllib.parse import urlparse

def get_civitai_model_id(civitai_url):
    try:
        model_id = re.search(r"models/(\d+)", civitai_url).group(1)
        if not model_id:
            print("No Model ID found in the download links.\n")
            return None

        print(f"Model ID successfully obtained: {model_id}\n")
        return model_id

    except Exception as e:
        print(f"An unexpected error occurred during the model setup: {str(e)}\n")
        print("If you encounter any issues, please copy this console output and open an issue at: https://github.com/deforum-studio/deforum/issues")


def instantiate_deforum_pipeline(models_path):

    # Start the timer
    start_time = time.time()

    model_map = {
        "SDXL-Base v1.0": {"url": "https://civitai.com/models/101055?modelVersionId=128078"},
        "SDXL-ProtoVisonXL_0630": {"url": "https://civitai.com/models/125703?modelVersionId=201514"},
        "SDXL-DynaVisionXL_0557": {"url": "https://civitai.com/models/122606?modelVersionId=198962"},
        "SDXL-NightVisionXL_0770": {"url": "https://civitai.com/models/128607?modelVersionId=201643"},
        "SDXL-ZavyChromaXL_v2.1": {"url": "https://civitai.com/models/119229?modelVersionId=169740"},
        "SDXL-JuggernautXL_v6": {"url": "https://civitai.com/models/133005?modelVersionId=198530"},
        "SDXL-UnstableDiffusers_YamerMix_v8": {"url": "https://civitai.com/models/84040?modelVersionId=196039"},
        "SDXL-Copax_TimelessXL_v8": {"url": "https://civitai.com/models/118111?modelVersionId=198246"},
        "SD_1.5-RealisticVision_v5.1": {"url": "https://civitai.com/models/4201?modelVersionId=130072"},
        "SD_1.5-Dreamshaper_v8": {"url": "https://civitai.com/models/4384/dreamshaper?modelVersionId=128713"},
        "SD_1.5-RevAnimated_v.1.2.2": {"url": "https://civitai.com/models/7371?modelVersionId=46846"},
        "SD_1.5-GhostMix_v2": {"url": "https://civitai.com/models/36520?modelVersionId=76907"}
    }

    try:
        model_checkpoint =  "SDXL-ProtoVisonXL_0630" #@param ['(model_path_or_url)', 'SDXL-Base v1.0', 'SDXL-ProtoVisonXL_0630', 'SDXL-DynaVisionXL_0557', 'SDXL-NightVisionXL_0770', 'SDXL-ZavyChromaXL_v2.1', 'SDXL-JuggernautXL_v6', 'SDXL-UnstableDiffusers_YamerMix_v8', 'SDXL-Copax_TimelessXL_v8', 'SD_1.5-RealisticVision_v5.1', 'SD_1.5-Dreamshaper_v8', 'SD_1.5-RevAnimated_v.1.2.2', 'SD_1.5-GhostMix_v2']
        model_path_or_url = '' #@param {type:"string"}

        if model_checkpoint == "(model_path_or_url)":
            if os.path.isfile(model_path_or_url):
                print("Loading local model file.\n")
                deforum = DeforumAnimationPipeline.from_single_file(model_path_or_url)
            else:
                parse_result = urlparse(model_path_or_url)
                if parse_result.scheme and parse_result.netloc:
                    print("Loading model from URL.\n")
                    model_id = get_civitai_model_id(model_path_or_url)
                    try:
                        deforum = DeforumAnimationPipeline.from_civitai(modelid=str(model_id), cache_dir=models_path)
                    except Exception as e:
                        print(f"Error loading model with from_civitai ({str(e)}), trying from_download_url.\n")
                        deforum = DeforumAnimationPipeline.from_download_url(model_path_or_url, cache_dir=models_path)
                else:
                    print(f"model_checkpoint: {model_checkpoint}")
                    print(f"model_path_or_url: {model_path_or_url}")
                    print("The model_path_or_url you provided is neither a valid local file nor a URL.\nPlease try again or select a model from the dropdown.")
                    print("If you believe this is an bug, please copy this console output and open an issue at: https://github.com/deforum-studio/deforum/issues")
        else:
            model_path_or_url = model_map[model_checkpoint]["url"]
            model_id = get_civitai_model_id(model_path_or_url)
            try:
                deforum = DeforumAnimationPipeline.from_civitai(modelid=str(model_id), cache_dir=models_path)
            except Exception as e:
                print(f"Error loading model with from_civitai ({str(e)})")

    except Exception as e:
        print(f"An unexpected error occurred during the model setup: {str(e)}")
        print("If you encounter any issues, please copy this console output and open an issue at: https://github.com/deforum-studio/deforum/issues")

    # Report the total time
    total_time = time.time() - start_time
    print('Model setup completed successfully. Total time: {:.2f} seconds'.format(total_time))
    return deforum

deforum = instantiate_deforum_pipeline(models_path)

## Run

In [ ]:
data = {"W": 1024, "H": 1024}
deforum(**data)